In [1]:
# add to path
import sys
if '../' not in sys.path:
    sys.path.insert(0, '../')
sys.path

['../',
 '/home/jimmyyao/repos/rev-stable-diffusion-2/notebooks',
 '/home/jimmyyao/miniconda3/envs/rev-sd2/lib/python39.zip',
 '/home/jimmyyao/miniconda3/envs/rev-sd2/lib/python3.9',
 '/home/jimmyyao/miniconda3/envs/rev-sd2/lib/python3.9/lib-dynload',
 '',
 '/home/jimmyyao/.local/lib/python3.9/site-packages',
 '/home/jimmyyao/miniconda3/envs/rev-sd2/lib/python3.9/site-packages',
 '/home/jimmyyao/repos/segment-anything',
 '/home/jimmyyao/miniconda3/envs/rev-sd2/lib/python3.9/site-packages/certifi-2022.12.7-py3.9.egg',
 '/home/jimmyyao/miniconda3/envs/rev-sd2/lib/python3.9/site-packages/wheel-0.40.0-py3.9.egg',
 '/home/jimmyyao/miniconda3/envs/rev-sd2/lib/python3.9/site-packages/setuptools-67.6.1-py3.9.egg',
 '/home/jimmyyao/repos/GenerativeImage2Text']

In [2]:
# param = {}

# image_encoder = get_image_encoder(
#         param.get('image_encoder_type', 'CLIPViT_B_16'),
#         input_resolution=param.get('test_crop_size', 224),
#     )

In [3]:
# i1 = torch.randn(2, 3, 120, 120)

# image_encoder(i1).shape

In [4]:
# t1 = torch.rand(768)
# t2 = torch.rand(768)
# t3 = torch.rand(768)

# li = [t1, t2, t3]
# # convert li to tensor
# t = torch.stack(li)
# t.shape

In [7]:
from generativeimage2text.train import get_image_transform, get_transform_image_norm, get_inception_train_transform, get_data, collate_fn, recursive_to_device
from generativeimage2text.common import Config

from generativeimage2text.model import get_image_encoder

from transformers import BertTokenizer

from generativeimage2text.layers.decoder import (TransformerDecoderTextualHead,
                             AutoRegressiveBeamSearch, GeneratorWithBeamSearch, create_projecton_layer)

import torch
import torch.nn as nn
import logging

from segment_anything import sam_model_registry, SamAutomaticMaskGenerator
from torch.cuda.amp import autocast
    
from samgit.decoder import SAMGitCaptioningModel
import cv2

def sam_image_crops(image, mask_generator, top_n_bbox):
    masks = mask_generator.generate(image)
    sorted_anns = sorted(masks, key=(lambda x: x['predicted_iou']), reverse=True)[:top_n_bbox]
    
    crops = []
    for ann in sorted_anns:
        x0, y0, x1, y1 = ann['bbox']
        if x0 > x1: 
            x0, x1 = x1, x0
        if y0 > y1: 
            y0, y1 = y1, y0

        crops.append(image[y0:y1, x0:x1, :])
    return crops

class ImgObjEncoder(nn.Module):
    def __init__(self, image_encoder, top_n_bbox=4, visual_feature_size=768, textual_feature_size=768, visual_projection_type='linearLn'):
        super().__init__()
        self.image_encoder = image_encoder
        self.top_n_bbox = top_n_bbox
        self.object_projection = create_projecton_layer(
            visual_projection_type='linearLn',
            visual_feature_size=visual_feature_size + visual_feature_size*top_n_bbox,
            textual_feature_size=textual_feature_size,
        )
        self.textual_feature_size = textual_feature_size
    
    def forward(self, batch, batch_image_crops: list[torch.Tensor]):
        # batch_image_crops = [batch, top_n_bbox, 3, 14, 14]
        batch_size = len(batch)
        image_features = self.image_encoder(batch)
        
        for b in range(len(batch_image_crops)): # batch
            batch_image_crop_features = self.image_encoder(batch_image_crops[b])
            top_n = len(batch_image_crops[b])
            print("batch_image_crop_features", batch_image_crop_features.shape)
            if top_n < self.top_n_bbox:
                padding = torch.zeros(self.top_n_bbox - top_n, self.object_feature_size)
                batch_image_crop_features = torch.cat([batch_image_crop_features, padding], dim=0)
            crops = []
            for i in range(len(image_crops[b])):
                crop = image_crops[b][i]
                if self.image_transform is not None:
                    crop = self.image_transform(crop)
                crops.append(crop)
            crops = torch.stack(crops)
            self.image_transform(crops)
            print("crops", crops.shape)
            
            object_features = self.object_projection(self.image_encoder(crop))
            batch_object_features.append(object_features)
        
        # list ot tensor
        padding = torch.stack(batch_padding, dim=0)
        object_features = torch.stack(batch_object_features, dim=0)
        # concat
        
        features = torch.cat([padding, image_features, object_features], dim=1)
        return features


def get_samgit_model(tokenizer, param):
    image_encoder = get_image_encoder(
        param.get('image_encoder_type', 'CLIPViT_B_16'),
        input_resolution=param.get('test_crop_size', 224),
    )
    
    img_obj_encoder = ImgObjEncoder(
        image_encoder,
        param.get('top_n_bbox', 4),
        visual_feature_size=param.get('visual_feature_size', 768),
        textual_feature_size=768,
        visual_projection_type=param.get('visual_projection_type', 'linearLn'),
    )
    text_decoder = TransformerDecoderTextualHead(
        visual_feature_size=param.get('visual_feature_size', 768),
        vocab_size=30522,
        hidden_size=768,
        num_layers=6,
        attention_heads=12,
        feedforward_size=768* 4,
        max_caption_length=1024,
        mask_future_positions=True,
        padding_idx=0,
        decoder_type='bert_en',
        visual_projection_type='linearLn',
    )
    decoder = GeneratorWithBeamSearch(
        eos_index=tokenizer.sep_token_id,
        #max_steps=40,
        max_steps=1024,
        beam_size=4,
        length_penalty=0.6,
    )
    
    model = SAMGitCaptioningModel(
        img_obj_encoder,
        text_decoder,
        decoder=decoder,
        sos_index=tokenizer.cls_token_id,
        eos_index=tokenizer.sep_token_id,
        tokenizer=tokenizer,
        use_history_for_infer=True,
        loss_type='smooth',
        num_image_with_embedding=param.get('num_image_with_embedding')
    )
    return model

def get_small_transform_vit_default(cfg, crop=14):
    default_normalize = transforms.Normalize(
            mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    normalize = get_transform_image_norm(cfg, default_normalize)
    transform = get_inception_train_transform(
        bgr2rgb=True,
        crop_size=crop,
        normalize=normalize,
        small_scale=cfg.input_small_scale,
        no_color_jitter=cfg.no_color_jitter,
        no_flip=cfg.no_flip,
        no_aspect_dist=cfg.no_aspect_dist,
        resize_crop=cfg.resize_crop,
        max_size=cfg.train_max_size,
        interpolation=cfg.interpolation or Image.BILINEAR,
    )
    return transform

import torchvision.transforms as transforms
from generativeimage2text.process_image import load_image_by_pil
def forward_backward(image_files, captions, prefixs=None):
    if prefixs is None:
        prefixs = [''] * len(captions)
    cfg = {
        'crop_region_extend_in_datatransform': 4,
        'data_normalize': 'clip',
        'train_crop_size': 224,
        'input_small_scale': 0.8,
        'no_color_jitter': True,
        'no_flip': True,
        'no_aspect_dist': True,
        'interpolation': 'bicubic',
        'min_size_range32': [160, 224], # in pretraining, it is multi-scale from 160 to 224; while for fine-tuning, it is single scale
        # 'min_size_range32': [224], # in pretraining, it is multi-scale from 160 to 224; while for fine-tuning, it is single scale
        'patch_size': 16,
        'train_transform': 'vitp',
    }
    cfg = Config(cfg, {})
    all_data = []
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
    image_transform = get_image_transform(cfg)
    for image_file, prefix, target in zip(image_files, prefixs, captions):
        data = get_data(image_file, prefix, target,
                        tokenizer, image_transform)
        all_data.append(data)
    data = collate_fn(all_data)
    data = recursive_to_device(data, 'cuda')
    
    param = {
        "top_n_bbox": 4,
    }
    model = get_samgit_model(tokenizer, param)
    model.half()
    model.train()
    model.cuda()
    
    sam = sam_model_registry[param.get("sam_model_type", "vit_h")](checkpoint=param.get("sam_checkpoint", "../models/sam_vit_h_4b8939.pth")).half()
    sam.half()
    sam.eval()
    sam.requires_grad_(False)
    sam.to(device='cuda')
    mask_generator = SamAutomaticMaskGenerator(
        model=sam,
        points_per_side=4,
        pred_iou_thresh=0.95,
        stability_score_thresh=0.95,
    )
    
    # resize to 14x14
    small_transform = get_small_transform_vit_default(cfg, crop=14)
    
    print("cuda util:",torch.cuda.utilization())
    with autocast():
        batch_img_crops = []
        for img_name in image_files:
            image = load_image_by_pil(img_name)
            # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            crops = sam_image_crops(image, mask_generator, param.get('top_n_bbox', 4))
            
            print(crops[0].shape)
            print(type(crops[0]))
            crops = small_transform(crops)
            crops = torch.stack(crops)
            batch_img_crops.append(crops)
            
        loss_dict = model(data, batch_img_crops)
        
    loss = sum(loss_dict.values())
    loss.backward()
    logging.info(loss)

ModuleNotFoundError: No module named 'generativeimage2text.preprocess_image'

In [ ]:
import pandas as pd
prompt_df = pd.read_csv('../datasets/sampleeval/prompts.csv')
prompt_df

,imgId,prompt
0,20057f34d,hyper realistic photo of very friendly and dys...
1,227ef0887,"ramen carved out of fractal rose ebony, in the..."
2,92e911621,ultrasaurus holding a black bean taco in the w...
3,a4e1c55a9,a thundering retro robot crane inks on parchme...
4,c98f79f71,"portrait painting of a shimmering greek hero, ..."
5,d8edf2e40,an astronaut standing on a engaging white rose...
6,f27825b2c,Kaggle employee Phil at a donut shop ordering ...


In [ ]:
image_files = [f"../datasets/sampleeval/{img}.png" for img in prompt_df['imgId']]
captions = [cap for cap in prompt_df['prompt']]

print(image_files)
print(captions)

['../datasets/sampleeval/20057f34d.png', '../datasets/sampleeval/227ef0887.png', '../datasets/sampleeval/92e911621.png', '../datasets/sampleeval/a4e1c55a9.png', '../datasets/sampleeval/c98f79f71.png', '../datasets/sampleeval/d8edf2e40.png', '../datasets/sampleeval/f27825b2c.png']
['hyper realistic photo of very friendly and dystopian crater', 'ramen carved out of fractal rose ebony, in the style of hudson river school', 'ultrasaurus holding a black bean taco in the woods, near an identical cheneosaurus', 'a thundering retro robot crane inks on parchment with a droopy french bulldog', 'portrait painting of a shimmering greek hero, next to a loud frill-necked lizard', 'an astronaut standing on a engaging white rose, in the midst of by ivory cherry blossoms', 'Kaggle employee Phil at a donut shop ordering all the best donuts, with a speech bubble that proclaims "Donuts. It\'s what\'s for dinner!"']


In [ ]:
forward_backward(image_files, captions)

: 

: 